<a href="https://colab.research.google.com/github/https://github.com/BhaskarS1ngha/ICPR_LEAF_INSPECT_CHALLENGE/blob/master/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEFINE SOURCE DIRECTORIES

In [ ]:
SOURCE_DIRECTORY = "Test_data"
MODEL_DIRECTORY = "mask2former-swin-large-leaf"
results_dir = "results"

# INSTALL DEPENDENCIES

In [ ]:
!pip install transformers albumentations opencv-python datasets==2.15


# LOAD INFERENCE DATA

In [ ]:
import pandas as pd
import numpy as np
import os




images = os.listdir(SOURCE_DIRECTORY)

#filter images with .png extension
images = [img for img in images if img.split('.')[-1] == 'png']



print('Total images:', len(images))


# create dataframe with columns img_no, label, img_path, mask_path
df = pd.DataFrame(columns=['img_no', 'img_path'])
imgNo = 0
for i in range(len(images)):
    img_path = os.path.join(SOURCE_DIRECTORY, images[i])
    tempDict = {'img_no': imgNo,
                'img_path': img_path}
    df = pd.concat([df, pd.DataFrame([tempDict])], ignore_index=True)
    imgNo += 1

#set index to img_no
df.set_index('img_no', inplace=True)
df.to_csv("dataset_validation.csv")

In [ ]:
import datasets

dataset_csv = "dataset_validation.csv"
dataset_df = pd.read_csv(dataset_csv)
dataset_df.head()

In [ ]:
dataset_df['img_name'] = dataset_df['img_path'].apply(lambda x: x.split(os.sep)[-1])
dataset_df.head()

In [ ]:
import os
IMAGES = dataset_df['img_path']
IMG_NAMES = dataset_df['img_name']
for i in range(len(IMAGES)):
    if not os.path.exists(IMAGES[i]):
        print(f"Image: {IMAGES[i]} does not exist")
        raise FileNotFoundError("Image does not exist")


In [ ]:
dataset = datasets.Dataset.from_dict({"pixel_values": IMAGES}, features=datasets.Features({"pixel_values": datasets.Image()}))

In [ ]:
dataset = dataset.add_column("img_name", IMG_NAMES)
dataset

In [ ]:
dataset[0]['pixel_values']

# INFERENCE

In [ ]:
from transformers import Mask2FormerForUniversalSegmentation
from transformers import MaskFormerImageProcessor
import torch

id2label = {0: "leaf"}
label2id = {"leaf": 0}
model = Mask2FormerForUniversalSegmentation.from_pretrained(MODEL_DIRECTORY, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)
processor = MaskFormerImageProcessor()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
if not os.path.exists(results_dir):
    os.mkdir(results_dir)

In [ ]:
import numpy as np
import cv2 as cv
from PIL import Image
def get_mask(segmentation, segment_id):
  mask = (segmentation.cpu().numpy() == segment_id)
  visual_mask = (mask * (segment_id+1)).astype(np.uint8)
  visual_mask = Image.fromarray(visual_mask)

  return visual_mask

## GENERATE OUTPUT

In [ ]:
n_imgs = len(dataset)
print(f"Processing {n_imgs} images")
for i in range(len(dataset)):
    image = dataset[i]["pixel_values"]
    img_name = dataset[i]["img_name"]
    print(f"Processing {i}/{n_imgs}: {img_name}")
    inputs = processor(image, return_tensors="pt").to(device)
    for k,v in inputs.items():
      print(k,v.shape)
    with torch.no_grad():
      outputs = model(**inputs)
    results = processor.post_process_instance_segmentation(outputs, target_sizes=[image.size[::-1]])[0]
    print(results.keys())
    fmask = None
    for segment in results['segments_info']:
        # print("Visualizing mask for instance:", model.config.id2label[segment['label_id']])
        mask = get_mask(results['segmentation'], segment['id'])
        if fmask is None:
          fmask = np.array(mask)
        fmask = cv.bitwise_or(fmask, np.array(mask))
        # display(mask)
    outPath = os.path.join(results_dir, img_name)
    cv.imwrite(outPath, fmask)
    print(f"Saved mask to {outPath}")